In [ ]:
import os
import json
import subprocess
from datetime import datetime
import shutil

# Ruta base del proyecto
base_path = r"C:\Users\Administrador\Documents\PythonScripts\Tesis\tesisaustral"

# Validar la ruta base
if not os.path.isdir(base_path):
    raise ValueError(f"La ruta proporcionada no es válida: {base_path}")

# Leer archivo de configuración
config_path = os.path.join(base_path, "config.json")
if not os.path.isfile(config_path):
    raise FileNotFoundError(f"No se encontró el archivo de configuración: {config_path}")

with open(config_path, "r") as config_file:
    config = json.load(config_file)

# Parámetros generales
scripts_path = os.path.join(base_path, config["paths"]["scripts"])
outputs_path = os.path.join(base_path, config["paths"]["outputs"])
experiment_name = "NRO_1"  # Nombre del experimento
timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

# Crear carpeta del experimento
experiment_folder = os.path.join(outputs_path, "experiments", f"{experiment_name}_{timestamp}")
os.makedirs(experiment_folder, exist_ok=True)

# Crear archivo de log
log_file = os.path.join(experiment_folder, f"log_{timestamp}.txt")

# Guardar una copia del archivo de configuración en la carpeta del experimento
shutil.copy(config_path, os.path.join(experiment_folder, "config.json"))

# Función para ejecutar notebooks
def run_notebook(notebook_path, output_name):
    output_file = os.path.join(experiment_folder, f"{output_name}.ipynb")
    command = [
        "jupyter", "nbconvert", "--to", "notebook", "--execute",
        "--output", output_file, notebook_path
    ]
    start_time = datetime.now()
    try:
        subprocess.run(command, check=True, stderr=subprocess.PIPE, stdout=subprocess.PIPE)
        end_time = datetime.now()
        with open(log_file, "a") as log:
            log.write(f"Ejecutado: {notebook_path} - Inicio: {start_time}, Fin: {end_time}\n")
        return output_file
    except subprocess.CalledProcessError as e:
        end_time = datetime.now()
        error_message = e.stderr.decode('utf-8') if e.stderr else "Sin detalles adicionales"
        with open(log_file, "a") as log:
            log.write(f"Error en: {notebook_path} - Inicio: {start_time}, Fin: {end_time}, Error: {error_message}\n")
        print(f"[ERROR] Error al ejecutar {notebook_path}: {error_message}")
        raise
    except FileNotFoundError as e:
        with open(log_file, "a") as log:
            log.write(f"[ERROR] El comando 'jupyter' no está disponible o no está instalado: {str(e)}\n")
        print(f"[ERROR] El comando 'jupyter' no está disponible o no está instalado: {str(e)}")
        raise

# Inicializar el archivo log
with open(log_file, "a") as log:
    log.write(f"Inicio del experimento: {experiment_name} - {timestamp}\n")

# Lista de notebooks a ejecutar (en orden)
notebooks = [
    "01_LecturayAnalisis.ipynb",
    "02_Normalization.ipynb",
    "03_Feateng_selection.ipynb",
    "04_Train_Test.ipynb"
]

# Ejecutar cada notebook
for notebook in notebooks:
    notebook_path = os.path.join(scripts_path, "notebooks", notebook)
    if not os.path.isfile(notebook_path):
        error_message = f"[ERROR] No se encontró el archivo del notebook: {notebook_path}"
        print(error_message)
        with open(log_file, "a") as log:
            log.write(f"{error_message}\n")
        continue

    output_name = f"output_{notebook.split('.')[0]}"
    try:
        print(f"[INFO] Ejecutando: {notebook}")
        with open(log_file, "a") as log:
            log.write(f"[INFO] Ejecutando: {notebook}\n")
        output_file = run_notebook(notebook_path, output_name)
        print(f"[INFO] Completado: {notebook}. Salida guardada en {output_file}")
        with open(log_file, "a") as log:
            log.write(f"[INFO] Completado: {notebook}. Salida guardada en {output_file}\n")
    except Exception as e:
        print(f"[ERROR] Error al ejecutar {notebook}: {e}")
        with open(log_file, "a") as log:
            log.write(f"[ERROR] Error al ejecutar {notebook}: {e}\n")

print(f"Proceso completado. Log guardado en {log_file}")